In [ ]:
# Path to tensorflow model research directory for yamnet
# https://github.com/tensorflow/models/tree/master/research/audioset
yamnet_path = '/home/ubuntu/models/research/audioset/yamnet/'


In [ ]:
import os
os.sys.path.insert(0, yamnet_path)
from params import Params as params
import features as features_lib

import numpy as np
import resampy
import soundfile as sf
import tensorflow as tf

# Directory with wav files for each class
train_dir = './train_set/'
# Directory that will store the spectrogram inputs for each class
out_dir = './train_set_patches/'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    for i in os.listdir(train_dir):
        os.mkdir(out_dir+i)
        

In [ ]:
for cls in os.listdir(train_dir):
    print(cls)
    for smp in os.listdir(train_dir+cls):
        
        wav_data, sr = sf.read(train_dir+cls+'/'+smp, dtype=np.int16)
        assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
        waveform = wav_data / 32768.0
        
        if len(waveform.shape) > 1:
            waveform = waveform[:,0] # get left channel if multi-channel
        if sr != params.sample_rate:
            waveform = resampy.resample(waveform, sr, params.sample_rate)
            
        waveform = np.reshape(waveform, [1, -1]).astype(np.float32)
        _, patches = features_lib.waveform_to_log_mel_spectrogram_patches(tf.squeeze(waveform, axis=0), params)
        
        np.save(out_dir+cls+'/'+smp+'.npy', patches)
        